# Analyse Exploratoire - Historique Spotify (2014-2024)

Ce notebook effectue une analyse exhaustive de l'historique d'écoute Spotify.
Les données couvrent environ 10 ans d'écoutes (novembre 2014 à septembre 2024).

## 1. Configuration et Connexion

Import des bibliothèques et connexion à la base de données PostgreSQL.

In [ ]:
# Imports principaux
import logging

import pandas as pd
import plotly.express as px
from sqlalchemy import create_engine

# Configuration du logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%H:%M:%S",
)
logger = logging.getLogger(__name__)

# Configuration de l'affichage
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

# Template Plotly pour un style cohérent
TEMPLATE = "plotly_white"

In [ ]:
# Connexion à la base de données PostgreSQL
# Charge l'URL depuis les variables d'environnement ou .env
import os
from pathlib import Path

# Charger le fichier .env si python-dotenv est disponible
try:
    from dotenv import load_dotenv

    env_path = (
        Path(__file__).parent.parent / ".env"
        if "__file__" in dir()
        else Path("../.env")
    )
    load_dotenv(env_path)
except ImportError:
    pass  # python-dotenv non installé, utiliser les variables d'environnement système

# Récupérer l'URL de connexion
DATABASE_URL = os.getenv(
    "DATABASE_URL",
    "postgresql://thomasbergot@localhost:5432/spotify_db",  # Valeur par défaut Homebrew
)

engine = create_engine(DATABASE_URL)

# Test de connexion (échouera explicitement si PostgreSQL n'est pas lancé)
with engine.connect() as conn:
    logger.info("Connexion réussie à PostgreSQL")

In [ ]:
# Chargement des données depuis PostgreSQL
# Requête complète sur la table streams
query = "SELECT * FROM streaming_history.streams"
df = pd.read_sql(query, engine)

logger.info(f"Nombre total d'enregistrements: {len(df):,}")
logger.info(f"Colonnes: {list(df.columns)}")

In [ ]:
# Préparation des données temporelles
# Conversion du timestamp (timezone-aware depuis PostgreSQL) et extraction des composantes
df["ts"] = pd.to_datetime(df["ts"], utc=True)
df["year"] = df["ts"].dt.year
df["month"] = df["ts"].dt.month
df["month_name"] = df["ts"].dt.month_name()
df["day_of_week"] = df["ts"].dt.day_name()
df["hour"] = df["ts"].dt.hour
df["date"] = df["ts"].dt.date

# Conversion de la durée en minutes pour plus de lisibilité
df["minutes_played"] = df["ms_played"] / 60000
df["hours_played"] = df["ms_played"] / 3600000

logger.info("Colonnes temporelles ajoutées avec succès")

## 2. Vue d'ensemble des données

Statistiques générales et aperçu de l'ensemble du dataset.

In [ ]:
# Statistiques générales
logger.info("=" * 50)
logger.info("STATISTIQUES GÉNÉRALES")
logger.info("=" * 50)
logger.info(f"Nombre total de streams: {len(df):,}")
logger.info(f"Période couverte: {df['ts'].min().date()} à {df['ts'].max().date()}")
logger.info(f"Durée totale d'écoute: {df['hours_played'].sum():,.0f} heures")
logger.info(f"Durée moyenne par stream: {df['minutes_played'].mean():.1f} minutes")
logger.info(f"Artistes uniques: {df['artist_name'].nunique():,}")
logger.info(f"Pistes uniques: {df['track_name'].nunique():,}")
logger.info(f"Albums uniques: {df['album_name'].nunique():,}")

In [ ]:
# Distribution de la durée d'écoute par stream
# Filtrage des valeurs aberrantes (> 60 minutes)
df_filtered = df[df["minutes_played"] <= 60]

fig = px.histogram(
    df_filtered,
    x="minutes_played",
    nbins=60,
    title="Distribution de la durée d'écoute par stream",
    labels={"minutes_played": "Durée (minutes)", "count": "Nombre de streams"},
    template=TEMPLATE,
    color_discrete_sequence=["#1DB954"],
)
fig.update_layout(bargap=0.1)
fig.show()

## 3. Analyse temporelle

Évolution des habitudes d'écoute dans le temps.

In [ ]:
# Évolution du nombre d'écoutes par année
yearly_streams = df.groupby("year").size().reset_index(name="count")

fig = px.bar(
    yearly_streams,
    x="year",
    y="count",
    title="Nombre d'écoutes par année",
    labels={"year": "Année", "count": "Nombre d'écoutes"},
    template=TEMPLATE,
    color_discrete_sequence=["#1DB954"],
)
fig.update_layout(xaxis={"tickmode": "linear"})
fig.show()

In [ ]:
# Évolution du temps d'écoute total par année (en heures)
yearly_hours = df.groupby("year")["hours_played"].sum().reset_index()

fig = px.bar(
    yearly_hours,
    x="year",
    y="hours_played",
    title="Temps d'écoute total par année",
    labels={"year": "Année", "hours_played": "Heures d'écoute"},
    template=TEMPLATE,
    color_discrete_sequence=["#1DB954"],
)
fig.update_layout(xaxis={"tickmode": "linear"})
fig.show()

In [ ]:
# Distribution par mois (saisonnalité)
month_order = [
    "January",
    "February",
    "March",
    "April",
    "May",
    "June",
    "July",
    "August",
    "September",
    "October",
    "November",
    "December",
]
monthly_streams = df.groupby("month_name").size().reset_index(name="count")
monthly_streams["month_name"] = pd.Categorical(
    monthly_streams["month_name"], categories=month_order, ordered=True
)
monthly_streams = monthly_streams.sort_values("month_name")

fig = px.bar(
    monthly_streams,
    x="month_name",
    y="count",
    title="Nombre d'écoutes par mois (toutes années confondues)",
    labels={"month_name": "Mois", "count": "Nombre d'écoutes"},
    template=TEMPLATE,
    color_discrete_sequence=["#1DB954"],
)
fig.show()

In [ ]:
# Distribution par jour de la semaine
day_order = [
    "Monday",
    "Tuesday",
    "Wednesday",
    "Thursday",
    "Friday",
    "Saturday",
    "Sunday",
]
daily_streams = df.groupby("day_of_week").size().reset_index(name="count")
daily_streams["day_of_week"] = pd.Categorical(
    daily_streams["day_of_week"], categories=day_order, ordered=True
)
daily_streams = daily_streams.sort_values("day_of_week")

fig = px.bar(
    daily_streams,
    x="day_of_week",
    y="count",
    title="Nombre d'écoutes par jour de la semaine",
    labels={"day_of_week": "Jour", "count": "Nombre d'écoutes"},
    template=TEMPLATE,
    color_discrete_sequence=["#1DB954"],
)
fig.show()

In [ ]:
# Distribution par heure de la journée
hourly_streams = df.groupby("hour").size().reset_index(name="count")

fig = px.bar(
    hourly_streams,
    x="hour",
    y="count",
    title="Nombre d'écoutes par heure de la journée",
    labels={"hour": "Heure", "count": "Nombre d'écoutes"},
    template=TEMPLATE,
    color_discrete_sequence=["#1DB954"],
)
fig.update_layout(xaxis={"tickmode": "linear", "dtick": 1})
fig.show()

In [ ]:
# Heatmap jour × heure
# Création d'un pivot table pour la heatmap
heatmap_data = df.groupby(["day_of_week", "hour"]).size().unstack(fill_value=0)
heatmap_data = heatmap_data.reindex(day_order)

fig = px.imshow(
    heatmap_data,
    labels={"x": "Heure", "y": "Jour", "color": "Écoutes"},
    title="Distribution des écoutes par jour et heure",
    template=TEMPLATE,
    color_continuous_scale="Greens",
    aspect="auto",
)
fig.update_layout(xaxis={"tickmode": "linear", "dtick": 1})
fig.show()

## 4. Analyse des artistes

Artistes les plus écoutés et diversité musicale.

In [ ]:
# Top 20 artistes les plus écoutés (nombre de streams)
top_artists = df["artist_name"].value_counts().head(20).reset_index()
top_artists.columns = ["artist", "count"]

fig = px.bar(
    top_artists,
    x="count",
    y="artist",
    orientation="h",
    title="Top 20 Artistes - Nombre de streams",
    labels={"count": "Nombre d'écoutes", "artist": "Artiste"},
    template=TEMPLATE,
    color_discrete_sequence=["#1DB954"],
)
fig.update_layout(yaxis={"categoryorder": "total ascending"})
fig.show()

In [ ]:
# Top 20 artistes par temps d'écoute total (heures)
artist_hours = (
    df.groupby("artist_name")["hours_played"].sum().nlargest(20).reset_index()
)
artist_hours.columns = ["artist", "hours"]

fig = px.bar(
    artist_hours,
    x="hours",
    y="artist",
    orientation="h",
    title="Top 20 Artistes - Temps d'écoute (heures)",
    labels={"hours": "Heures d'écoute", "artist": "Artiste"},
    template=TEMPLATE,
    color_discrete_sequence=["#1DB954"],
)
fig.update_layout(yaxis={"categoryorder": "total ascending"})
fig.show()

In [ ]:
# Diversité musicale par année (nombre d'artistes uniques)
diversity = df.groupby("year")["artist_name"].nunique().reset_index()
diversity.columns = ["year", "unique_artists"]

fig = px.line(
    diversity,
    x="year",
    y="unique_artists",
    title="Diversité musicale - Artistes uniques par année",
    labels={"year": "Année", "unique_artists": "Artistes uniques"},
    template=TEMPLATE,
    markers=True,
)
fig.update_traces(line_color="#1DB954")
fig.update_layout(xaxis={"tickmode": "linear"})
fig.show()

In [ ]:
# Évolution des top 5 artistes dans le temps
top5_artists = df["artist_name"].value_counts().head(5).index.tolist()
top5_yearly = (
    df[df["artist_name"].isin(top5_artists)]
    .groupby(["year", "artist_name"])
    .size()
    .reset_index(name="count")
)

fig = px.line(
    top5_yearly,
    x="year",
    y="count",
    color="artist_name",
    title="Évolution des 5 artistes les plus écoutés",
    labels={"year": "Année", "count": "Nombre d'écoutes", "artist_name": "Artiste"},
    template=TEMPLATE,
    markers=True,
)
fig.update_layout(xaxis={"tickmode": "linear"})
fig.show()

## 5. Analyse des pistes

Pistes les plus écoutées et comportement d'écoute.

In [ ]:
# Top 30 pistes les plus écoutées
# Création d'un label combinant artiste et piste pour plus de clarté
df["track_label"] = (
    df["artist_name"].fillna("Unknown") + " - " + df["track_name"].fillna("Unknown")
)
top_tracks = df["track_label"].value_counts().head(30).reset_index()
top_tracks.columns = ["track", "count"]

fig = px.bar(
    top_tracks,
    x="count",
    y="track",
    orientation="h",
    title="Top 30 Pistes les plus écoutées",
    labels={"count": "Nombre d'écoutes", "track": "Piste"},
    template=TEMPLATE,
    color_discrete_sequence=["#1DB954"],
)
fig.update_layout(yaxis={"categoryorder": "total ascending"}, height=800)
fig.show()

In [ ]:
# Analyse du taux de skip par piste (top 20 pistes les plus skippées)
# Filtrer sur les pistes avec au moins 50 écoutes pour avoir des résultats significatifs
# Convertir skipped en int (True=1, False=0, NaN=0) pour le calcul
df["skipped_int"] = df["skipped"].fillna(False).astype(int)

track_stats = (
    df.groupby("track_label")
    .agg(total_plays=("id", "count"), skipped_count=("skipped_int", "sum"))
    .reset_index()
)
track_stats = track_stats[track_stats["total_plays"] >= 50]
track_stats["skip_rate"] = (
    track_stats["skipped_count"] / track_stats["total_plays"] * 100
).round(1)
most_skipped = track_stats.nlargest(20, "skip_rate")

fig = px.bar(
    most_skipped,
    x="skip_rate",
    y="track_label",
    orientation="h",
    title="Top 20 Pistes les plus skippées (min. 50 écoutes)",
    labels={"skip_rate": "Taux de skip (%)", "track_label": "Piste"},
    template=TEMPLATE,
    color_discrete_sequence=["#E74C3C"],
)
fig.update_layout(yaxis={"categoryorder": "total ascending"})
fig.show()

In [ ]:
# Top 20 pistes avec le plus de skips (nombre absolu)
# Utile pour identifier les pistes souvent lancées mais rarement écoutées en entier
track_skips_absolute = (
    df.groupby("track_label")
    .agg(total_plays=("id", "count"), skipped_count=("skipped_int", "sum"))
    .reset_index()
)
track_skips_absolute = track_skips_absolute[track_skips_absolute["skipped_count"] > 0]
most_skipped_absolute = track_skips_absolute.nlargest(20, "skipped_count")

fig = px.bar(
    most_skipped_absolute,
    x="skipped_count",
    y="track_label",
    orientation="h",
    title="Top 20 Pistes les plus skippées (nombre absolu)",
    labels={"skipped_count": "Nombre de skips", "track_label": "Piste"},
    template=TEMPLATE,
    color_discrete_sequence=["#E74C3C"],
)
fig.update_layout(yaxis={"categoryorder": "total ascending"})
fig.show()

In [ ]:
# Top 20 artistes les plus skippés (par taux de skip)
# Filtrer sur les artistes avec au moins 100 écoutes pour avoir des résultats significatifs
artist_skip_stats = (
    df.groupby("artist_name")
    .agg(total_plays=("id", "count"), skipped_count=("skipped_int", "sum"))
    .reset_index()
)
artist_skip_stats = artist_skip_stats[artist_skip_stats["total_plays"] >= 100]
artist_skip_stats["skip_rate"] = (
    artist_skip_stats["skipped_count"] / artist_skip_stats["total_plays"] * 100
).round(1)
most_skipped_artists = artist_skip_stats.nlargest(20, "skip_rate")

fig = px.bar(
    most_skipped_artists,
    x="skip_rate",
    y="artist_name",
    orientation="h",
    title="Top 20 Artistes les plus skippés (min. 100 écoutes)",
    labels={"skip_rate": "Taux de skip (%)", "artist_name": "Artiste"},
    template=TEMPLATE,
    color_discrete_sequence=["#E74C3C"],
)
fig.update_layout(yaxis={"categoryorder": "total ascending"})
fig.show()

In [ ]:
# Top 20 artistes avec le plus de skips (nombre absolu)
artist_skips_absolute = (
    df.groupby("artist_name")
    .agg(total_plays=("id", "count"), skipped_count=("skipped_int", "sum"))
    .reset_index()
)
artist_skips_absolute = artist_skips_absolute[
    artist_skips_absolute["skipped_count"] > 0
]
most_skipped_artists_abs = artist_skips_absolute.nlargest(20, "skipped_count")

fig = px.bar(
    most_skipped_artists_abs,
    x="skipped_count",
    y="artist_name",
    orientation="h",
    title="Top 20 Artistes les plus skippés (nombre absolu)",
    labels={"skipped_count": "Nombre de skips", "artist_name": "Artiste"},
    template=TEMPLATE,
    color_discrete_sequence=["#E74C3C"],
)
fig.update_layout(yaxis={"categoryorder": "total ascending"})
fig.show()

## 6. Analyse des albums

Albums les plus écoutés.

In [ ]:
# Top 20 albums les plus écoutés
df["album_label"] = (
    df["artist_name"].fillna("Unknown") + " - " + df["album_name"].fillna("Unknown")
)
top_albums = df["album_label"].value_counts().head(20).reset_index()
top_albums.columns = ["album", "count"]

fig = px.bar(
    top_albums,
    x="count",
    y="album",
    orientation="h",
    title="Top 20 Albums les plus écoutés",
    labels={"count": "Nombre d'écoutes", "album": "Album"},
    template=TEMPLATE,
    color_discrete_sequence=["#1DB954"],
)
fig.update_layout(yaxis={"categoryorder": "total ascending"}, height=600)
fig.show()

## 7. Analyse des plateformes et appareils

Répartition des écoutes par plateforme utilisée.

In [ ]:
# Simplification des noms de plateformes
def simplify_platform(platform):
    if pd.isna(platform):
        return "Unknown"
    platform_lower = platform.lower()
    if (
        "ios" in platform_lower
        or "iphone" in platform_lower
        or "ipad" in platform_lower
    ):
        return "iOS"
    elif "android" in platform_lower:
        return "Android"
    elif (
        "os x" in platform_lower or "macos" in platform_lower or "osx" in platform_lower
    ):
        return "macOS"
    elif "windows" in platform_lower:
        return "Windows"
    elif "web" in platform_lower:
        return "Web Player"
    elif "linux" in platform_lower:
        return "Linux"
    else:
        return "Other"


df["platform_simple"] = df["platform"].apply(simplify_platform)

# Répartition des écoutes par plateforme
platform_counts = df["platform_simple"].value_counts().reset_index()
platform_counts.columns = ["platform", "count"]

fig = px.pie(
    platform_counts,
    values="count",
    names="platform",
    title="Répartition des écoutes par plateforme",
    template=TEMPLATE,
)
fig.show()

In [ ]:
# Évolution des plateformes dans le temps
platform_yearly = (
    df.groupby(["year", "platform_simple"]).size().reset_index(name="count")
)

fig = px.area(
    platform_yearly,
    x="year",
    y="count",
    color="platform_simple",
    title="Évolution des plateformes utilisées",
    labels={
        "year": "Année",
        "count": "Nombre d'écoutes",
        "platform_simple": "Plateforme",
    },
    template=TEMPLATE,
)
fig.update_layout(xaxis={"tickmode": "linear"})
fig.show()

## 8. Analyse géographique

Répartition des écoutes par pays de connexion.

In [ ]:
# Répartition par pays
country_counts = df["conn_country"].value_counts().head(15).reset_index()
country_counts.columns = ["country", "count"]

fig = px.bar(
    country_counts,
    x="country",
    y="count",
    title="Répartition des écoutes par pays (Top 15)",
    labels={"country": "Pays", "count": "Nombre d'écoutes"},
    template=TEMPLATE,
    color_discrete_sequence=["#1DB954"],
)
fig.show()

## 9. Comportement d'écoute

Analyse des modes de lecture et des patterns d'écoute.

In [ ]:
# Analyse du mode shuffle
# Conversion explicite pour gérer les valeurs nulles
shuffle_labels = df["shuffle"].apply(
    lambda x: "Activé" if x is True else ("Désactivé" if x is False else "Inconnu")
)
shuffle_counts = shuffle_labels.value_counts().reset_index()
shuffle_counts.columns = ["shuffle", "count"]

fig = px.pie(
    shuffle_counts,
    values="count",
    names="shuffle",
    title="Mode Shuffle",
    template=TEMPLATE,
    color_discrete_sequence=["#1DB954", "#191414", "#888888"],
)
fig.show()

In [ ]:
# Analyse du mode offline
# Conversion explicite pour gérer les valeurs nulles
offline_labels = df["offline"].apply(
    lambda x: "Hors-ligne" if x is True else ("En ligne" if x is False else "Inconnu")
)
offline_counts = offline_labels.value_counts().reset_index()
offline_counts.columns = ["offline", "count"]

fig = px.pie(
    offline_counts,
    values="count",
    names="offline",
    title="Mode de connexion",
    template=TEMPLATE,
    color_discrete_sequence=["#E74C3C", "#1DB954", "#888888"],
)
fig.show()

In [ ]:
# Analyse des raisons de début de lecture
start_reasons = df["reason_start"].value_counts().head(10).reset_index()
start_reasons.columns = ["reason", "count"]

fig = px.bar(
    start_reasons,
    x="count",
    y="reason",
    orientation="h",
    title="Raisons de début de lecture",
    labels={"count": "Nombre d'écoutes", "reason": "Raison"},
    template=TEMPLATE,
    color_discrete_sequence=["#1DB954"],
)
fig.update_layout(yaxis={"categoryorder": "total ascending"})
fig.show()

In [ ]:
# Analyse des raisons de fin de lecture
end_reasons = df["reason_end"].value_counts().head(10).reset_index()
end_reasons.columns = ["reason", "count"]

fig = px.bar(
    end_reasons,
    x="count",
    y="reason",
    orientation="h",
    title="Raisons de fin de lecture",
    labels={"count": "Nombre d'écoutes", "reason": "Raison"},
    template=TEMPLATE,
    color_discrete_sequence=["#1DB954"],
)
fig.update_layout(yaxis={"categoryorder": "total ascending"})
fig.show()

In [ ]:
# Taux de skip global
# Utiliser skipped_int créé précédemment pour éviter les problèmes avec NaN
if "skipped_int" not in df.columns:
    df["skipped_int"] = df["skipped"].fillna(False).astype(int)

skip_rate = df["skipped_int"].sum() / len(df) * 100
logger.info(f"Taux de skip global: {skip_rate:.1f}%")

# Évolution du taux de skip par année
yearly_skip = (
    df.groupby("year")
    .agg(total=("id", "count"), skipped=("skipped_int", "sum"))
    .reset_index()
)
yearly_skip["skip_rate"] = (yearly_skip["skipped"] / yearly_skip["total"] * 100).round(
    1
)

fig = px.line(
    yearly_skip,
    x="year",
    y="skip_rate",
    title="Évolution du taux de skip par année",
    labels={"year": "Année", "skip_rate": "Taux de skip (%)"},
    template=TEMPLATE,
    markers=True,
)
fig.update_traces(line_color="#E74C3C")
fig.update_layout(xaxis={"tickmode": "linear"})
fig.show()

In [ ]:
# Mode incognito
# Utiliser fillna pour remplacer NaN par False avant le calcul
incognito_count = df["incognito_mode"].fillna(False).sum()
incognito_pct = incognito_count / len(df) * 100

logger.info(
    f"Écoutes en mode incognito: {int(incognito_count):,} ({incognito_pct:.2f}%)"
)

## 10. Tendances et patterns avancés

Analyses complémentaires sur les habitudes d'écoute.

In [ ]:
# Découverte de nouveaux artistes par année
# Un artiste est "nouveau" s'il n'a jamais été écouté les années précédentes
df_sorted = df.sort_values("ts")
first_listen = df_sorted.groupby("artist_name")["year"].min().reset_index()
first_listen.columns = ["artist_name", "first_year"]
new_artists_per_year = (
    first_listen.groupby("first_year").size().reset_index(name="new_artists")
)

fig = px.bar(
    new_artists_per_year,
    x="first_year",
    y="new_artists",
    title="Découverte de nouveaux artistes par année",
    labels={"first_year": "Année", "new_artists": "Nouveaux artistes"},
    template=TEMPLATE,
    color_discrete_sequence=["#1DB954"],
)
fig.update_layout(xaxis={"tickmode": "linear"})
fig.show()

In [ ]:
# Fidélité aux artistes: artistes écoutés sur plusieurs années
artist_years = df.groupby("artist_name")["year"].nunique().reset_index()
artist_years.columns = ["artist", "years_active"]
loyalty_distribution = (
    artist_years["years_active"].value_counts().sort_index().reset_index()
)
loyalty_distribution.columns = ["years", "artists_count"]

fig = px.bar(
    loyalty_distribution,
    x="years",
    y="artists_count",
    title="Fidélité aux artistes (nombre d'années d'écoute)",
    labels={"years": "Nombre d'années", "artists_count": "Nombre d'artistes"},
    template=TEMPLATE,
    color_discrete_sequence=["#1DB954"],
)
fig.update_layout(xaxis={"tickmode": "linear", "dtick": 1})
fig.show()

In [ ]:
# Artistes les plus fidèles (écoutés le plus grand nombre d'années)
most_loyal = artist_years.nlargest(20, "years_active")

fig = px.bar(
    most_loyal,
    x="years_active",
    y="artist",
    orientation="h",
    title="Top 20 Artistes les plus fidèlement écoutés",
    labels={"years_active": "Nombre d'années d'écoute", "artist": "Artiste"},
    template=TEMPLATE,
    color_discrete_sequence=["#1DB954"],
)
fig.update_layout(yaxis={"categoryorder": "total ascending"})
fig.show()

## 11. Synthèse et conclusions

Résumé des insights clés de l'analyse.

In [ ]:
# Résumé textuel des insights
logger.info("=" * 60)
logger.info("RÉSUMÉ DES INSIGHTS CLÉS")
logger.info("=" * 60)

# Artiste favori
top_artist = df["artist_name"].value_counts().idxmax()
top_artist_count = df["artist_name"].value_counts().max()
logger.info(f"Artiste le plus écouté: {top_artist} ({top_artist_count:,} écoutes)")

# Piste favorite
top_track = df["track_label"].value_counts().idxmax()
top_track_count = df["track_label"].value_counts().max()
logger.info(f"Piste la plus écoutée: {top_track} ({top_track_count:,} écoutes)")

# Année la plus active
most_active_year = df["year"].value_counts().idxmax()
most_active_count = df["year"].value_counts().max()
logger.info(f"Année la plus active: {most_active_year} ({most_active_count:,} écoutes)")

# Heure préférée
favorite_hour = df["hour"].value_counts().idxmax()
logger.info(f"Heure d'écoute préférée: {favorite_hour}h")

# Jour préféré
favorite_day = df["day_of_week"].value_counts().idxmax()
logger.info(f"Jour d'écoute préféré: {favorite_day}")

# Plateforme principale
main_platform = df["platform_simple"].value_counts().idxmax()
logger.info(f"Plateforme principale: {main_platform}")

logger.info(f"Total: {len(df):,} streams sur {df['hours_played'].sum():,.0f} heures")
logger.info(f"Soit environ {df['hours_played'].sum()/365:.0f} heures par an en moyenne")